In [1]:
!pip install pandas


In [2]:
import pandas as pd
import os

In [3]:
%pwd

'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
import sys
print(sys.executable)
print(sys.version)
print(sys.path)


d:\python_project\DataScience\StoresSalesPrediction\BigMartStoreSalesPrediction\venv\Scripts\python.exe
3.9.0 (tags/v3.9.0:9cf6752, Oct  5 2020, 15:34:40) [MSC v.1927 64 bit (AMD64)]
['C:\\Users\\sruth\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip', 'C:\\Users\\sruth\\AppData\\Local\\Programs\\Python\\Python39\\DLLs', 'C:\\Users\\sruth\\AppData\\Local\\Programs\\Python\\Python39\\lib', 'C:\\Users\\sruth\\AppData\\Local\\Programs\\Python\\Python39', 'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction\\venv', '', 'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction\\venv\\lib\\site-packages', 'D:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction\\src', 'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction\\venv\\lib\\site-packages\\salesprediction-0.0.0-py3.9.egg', 'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction\\v

In [ ]:
from salesPrediction.constants import *
from salesPrediction.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):  
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL =  config.source_URL,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from salesPrediction import logger
from salesPrediction.utils.common import get_size

In [ ]:

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{header}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)



In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-02-18 13:20:34,613: common:yaml file: config\config.yaml loaded successfully]
[2025-02-18 13:20:34,633: common:yaml file: params.yaml loaded successfully]
[2025-02-18 13:20:34,658: common:yaml file: schema.yaml loaded successfully]
[2025-02-18 13:20:34,668: common:created directory at artifacts]
[2025-02-18 13:20:34,671: common:created directory at artifacts/data_ingestion]
[2025-02-18 13:20:38,898: 1086410102:artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F3F2:170632:24588F:2719B7:67B450E5
Accept-Ranges: bytes
Date: Tue, 18 Feb 2025 09:20:37 GMT
Via: 1.1 varnish
X-S